In [1]:
# 필요 라이브러리 임포트

import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import urllib3
import matplotlib.pyplot as plt

tf.config.list_physical_devices('GPU')
urllib3.disable_warnings()

tfds.disable_progress_bar()

In [2]:
# 데이터셋 다운로드
# tfds.load()는 기본적으로 ~/tensorflow_datasets 경로에 데이터셋을 다운
# 데이터셋 경로를 바꾸고 싶으시다면, data_dir 인자를 사용

(ds_train, ds_test), ds_info = tfds.load('cifar10', split = ['train', 'test'], shuffle_files = True, with_info = True)

In [3]:
# 다운받은 데이터셋 feature 정보 확인

print(ds_info.features)

FeaturesDict({
    'id': Text(shape=(), dtype=tf.string),
    'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
})


In [4]:
# 데이터의 개수 확인
# tensorflow 데이터셋은 numpy 가 아니기 때문에 ds_train.shape로 확인하지 않는다.

print(tf.data.experimental.cardinality(ds_train))
print(tf.data.experimental.cardinality(ds_test))

tf.Tensor(50000, shape=(), dtype=int64)
tf.Tensor(10000, shape=(), dtype=int64)


In [5]:
# input normalization
# 각 채널별 최대값인 255로 정규화

def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

def apply_normalize_on_dataset(ds, is_test = False, batch_size = 16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [6]:
# resnet - 34 블록 구성

def build_resnet_34_block(input_layer, num_cnn = 3, channel = 64, block_num = 1) :
    x = input_layer    # 입력 레이어
    
    # CNN 레이어
    for cnn_num in range(num_cnn) : 
        x = keras.layers.Conv2D(filters = channel, kernel_size = (3, 3), activation = 'relu', kernel_initializer = 'he_normal', padding = 'same', name = f'block{block_num}_conv{cnn_num}')(x)
      
    return x

In [7]:
# 입력 레이어 생성
resnet_34_input_layer = keras.layers.Input(shape = (32, 32, 3))

# resnet 34 블록 생성
resnet_34_block_out = build_resnet_34_block(resnet_34_input_layer)

In [8]:
# 블록 1개짜리 model 생성

model = keras.Model(inputs = resnet_34_input_layer, outputs = resnet_34_block_out)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv0 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
Total params: 75,648
Trainable params: 75,648
Non-trainable params: 0
_________________________________________________________________


In [11]:
# ResNet - 34 모델 생성

def build_resnet_34(input_layer, num_cnn = 3, channel = 64, block_num = 1) :
    x = input_layer
    
    output = keras.layers.Conv2D(filter = channel, kernel_size = (3, 3), )
    

In [ ]:
# ResNet - 34
# (3 x 3, 64)형태의 레이어가 1블록에 3, 4, 6, 3 개로 총 4개의 블록으로 구성

resnet_34 = build_resent(input_shape = (32, 32, 3), is_50 = False)

resent_34.summary()

In [ ]:
# 

In [ ]:
# ResNet - 50
# (1 x 1, 64), (3 x 3, 64), (1 x 1, 64) 형태의 레이어가 1블록에 

resnet_50 = build_resnet(input_shape = (32, 32, 3), is_50 = True)

resnet_50.summary()